# Práctico 9: medición de objetos
La siguiente imagen es una foto con diferentes objetos en un mismo plano. Sabiendo que los lados del cuadrado rojo (papel glace) miden 100mm.

<img src="ej9_referenciacion.png" alt="Drawing" style="width: 200px;"/>

A. Escribir un programa que en forma automática usando solo las medidas conocidas sea
capaz de medir algunas de las cosas siguientes:
* ancho y alto de la tarjeta,
* ancho y alto de la goma o
* radio de ambas monedas.

In [14]:
import cv2
import numpy as np

In [15]:
points1 = [[85, 167], [402, 219], [394, 527], [41, 506]]
i = 0

In [16]:
def Trans_perspectiva(image, img1):
    h, w = img1.shape[:2] 
    pts1 = np.float32([[points1[0]], [points1[1]], [points1[2]], [points1[3]]])
    print(pts1)
    pts2 = np.float32([[0,0], [350, 0], [350,350], [0, 350]])
    M = cv2.getPerspectiveTransform(pts1, pts2)
    final = cv2.warpPerspective(img1, M, (w,h))
    return final

In [25]:
img1 = cv2.imread('ej9_referenciacion.png', 1)
img2 = np.zeros((img1.shape[0], img1.shape[1], 3), np.uint8)

img1_aux = img1.copy()
img1_aux = cv2.putText( img1_aux , 'Pulse para medir:', ( 0, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2 , cv2.LINE_AA)
img1_aux = cv2.putText( img1_aux , '1-tarjeta; 2-goma; 3-moneda', ( 0, 65), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2 , cv2.LINE_AA)

# 350px son 100mm

while(1):

    cv2.imshow('Seleccionar que quiero medir', img1_aux)
    k = cv2.waitKey(0) & 0xFF
    img_transf = Trans_perspectiva(img2, img1)
    if k == ord('1'):
        #ancho... 195 pixeles = 5,57 cm
        img_transf = cv2.line(img_transf, (380,30), (575, 27), (255,0,0))
        img_transf = cv2.putText( img_transf , 'w=5,57cm', ( 410, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2 , cv2.LINE_AA)
        #alto... 300 pixeles = 8,57 cm
        img_transf = cv2.line(img_transf, (575,30), (575, 330), (255,0,0))
        img_transf = cv2.putText( img_transf , 'h=8,57cm', ( 425,165), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2 , cv2.LINE_AA)
        cv2.imwrite('Resultado.jpg', img_transf)
        cv2.namedWindow('Resultado')
        cv2.imshow('Resultado', img_transf) 

    if k == ord('2'):
        #ancho... 200 pixeles = 5,71 cm
        img_transf = cv2.line(img_transf, (64,413), (264,413), (255,0,0))
        img_transf = cv2.putText( img_transf , 'w=5,71cm', (100,413), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2 , cv2.LINE_AA)
        #alto... 67 pixeles = 1,91 cm
        img_transf = cv2.line(img_transf, (265,415), (265,482), (255,0,0))
        img_transf = cv2.putText( img_transf , 'h=1,91cm', (265,450), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2 , cv2.LINE_AA)
        cv2.imwrite('Resultado.jpg', img_transf)
        cv2.namedWindow('Resultado')
        cv2.imshow('Resultado', img_transf) 

    if k == ord('3'):
        img = img_transf.copy()
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.medianBlur(img,5)
        circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,20,
                                    param1=50,param2=30,minRadius=30,maxRadius=50)
        circles = np.uint16(np.around(circles))
        for i in circles[0,:]:
            # draw the outer circle
            cv2.circle(img_transf,(i[0],i[1]),i[2],(0,255,0),2)
            # draw the center of the circle
            cv2.circle(img_transf,(i[0],i[1]),2,(0,0,255),3)
            # draw medition line
            img_transf = cv2.line(img_transf, (i[0],i[1]), (i[0],i[1]-i[2]), (255,0,0))
            img_transf = cv2.putText( img_transf , 'r=' + str(i[2]*100/350) + 'mm', (i[0],i[1]-i[2]), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (255, 0, 0), 2 , cv2.LINE_AA)
        
        cv2.imwrite('Resultado.jpg', img_transf)
        cv2.namedWindow('Resultado')
        cv2.imshow('Resultado', img_transf) 
    elif k == ord('q'):
        break

cv2.waitKey(0)      
cv2.destroyAllWindows()


#puntos del cuadrado de glace [[[ 85. 169.]]  [[402. 219.]] [[396. 524.]] [[ 44. 504.]]]

[[[ 85. 167.]]

 [[402. 219.]]

 [[394. 527.]]

 [[ 41. 506.]]]
[[[ 85. 167.]]

 [[402. 219.]]

 [[394. 527.]]

 [[ 41. 506.]]]


In [26]:
img1 = cv2.imread('ej9_referenciacion.png', 1)
img2 = np.zeros((img1.shape[0], img1.shape[1], 3), np.uint8)
image = Trans_perspectiva(img2, img1)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (3, 3), 0)
canny = cv2.Canny(blurred, 50, 200, 3)

# Find contours
cnts = cv2.findContours(canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

# Iterate thorugh contours and draw rectangles around contours
for c in cnts[0:3]:
    x,y,w,h = cv2.boundingRect(c)
    cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 2)
    #ancho... 200 pixeles = 5,71 cm
    cv2.putText( image, 'w=' + str(w*100/350) + 'mm' + '/ h=' + str(h*100/350) + 'mm', (x+5,y+h/2), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2 , cv2.LINE_AA)
    #alto... 67 pixeles = 1,91 cm
    #cv2.putText(image, 'h=' + str(h) + 'mm', (265,450), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2 , cv2.LINE_AA)

cv2.imshow('canny', canny)
cv2.imshow('image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[[ 85. 167.]]

 [[402. 219.]]

 [[394. 527.]]

 [[ 41. 506.]]]


In [ ]:
lines